# Purpose

# Imports

In [1]:
import pandas as pd
import numpy as np
import logging
logger = logging.getLogger()

import os
os.chdir("..") # go up to root

import datetime as dt
import pytz

# SBA Paycheck Protection Program Data



## Load Latest Cleaned File

Other members of the volunteer data scientist team are doing data cleaning of their own and pushing updates into a shared Google Drive, so let's get that first!

In [15]:
from src.data.google_drive_quickstart import main

main()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [10]:
from google.auth.transport.requests import Request

In [6]:
import os

KEYS_PATH = '/home/jovyan/work/secure_keys/'
TOKEN_PATH = KEYS_PATH + 'token.pickle'

os.path.exists(TOKEN_PATH)

False

In [7]:
creds = None

not creds

True

In [9]:
print(creds and creds.expired and creds.refresh_token)

None
